<br>

# *Imports*
Inicialmente faz-se necessário criar uma pasta que receberão os dados do IBGE.

In [ ]:
import os
import json
import shutil
import folium
import urllib.request
import pandas as pd
import geopandas as gpd

In [ ]:
def predict_encoding(file_path, n_lines=30):
    """
    Predict Encoding determines the encoding of a given file, so that when reading the file (via 'Pandas', for example)
    characters, accents and symbols are spelled correctly.
    :param file_path: Name and path to file
    :param n_lines: Number of lines to read to set encoding. Parameter Optional. Default: 30 lines.
    :return: Encoding.
    """
    import chardet

    # Open the file as binary data
    with open(file_path, 'rb') as f:
        # Join binary lines for specified number of lines
        raw_data = b''.join([f.readline() for line in range(n_lines)])

    return chardet.detect(raw_data)['encoding']

In [ ]:
os.makedirs('data', exist_ok=True)

<br>

# Dados Espaciais

Com a estrutura de pastas criada, é possivel fazer o download dos arquivos disponiblizados pelo IBGE. Há uma infinidade de dados e ainda, há a <a title="Link da API" href="http://api.sidra.ibge.gov.br" target="_blank">**_API do SIDRA_**</a> que possibilita obter mais dados.

- https://servicodados.ibge.gov.br/api/docs

In [ ]:
# Define qual o código IBGE do Estado
estado = 35

# Define URL
url = ('https://servicodados.ibge.gov.br/api/v2/malhas/'+
       str(estado)+'/?'+
       '&formato=application/vnd.geo+json'+
       '&resolucao=5'+
       '&qualidade=4')


# Define o nome do arquivo que será salvo as informações do IBGE
filename = os.path.join('data', 'divadmin.json')

# Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
       shutil.copyfileobj(response, out_file)

In [ ]:
file_encoding = predict_encoding(filename)
file_encoding

In [ ]:
# Map Object
m = folium.Map(
    location=[-23.9619271,-46.3427499],    
    zoom_start=1,
)

# Folium Object
folium.GeoJson(
    filename,
    name='Municípios',
).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

Ou ainda, ao invés de baixar o arquivo, é possivel fazer com o que o mapa seja criado com a leitura dos dados diretamente do site do IBGE. Nessa função o *encoding* já foi definido, evitando o problema mencionado acima.

In [ ]:
web = urllib.request.urlopen(url)
data = web.read()
encoding = web.info().get_content_charset('utf-8')

jsn = json.loads(data.decode(encoding))
#jsn

In [ ]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=8
)

folium.GeoJson(
    jsn,
    name='Municípios',
).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

Uma vez com o mapa na mão, de qualquer que seja o meio que foi obtido, é possivel analisar a "tabela de atributos".
Lá descobrimos que existe o par de coordenadas que define o centroide e, ainda, o 'codarea' que tem o código do IBGE do município.

<br>

# Dados Tabulares

In [ ]:
# Defines IBGE Code State
estado = 35

# Defines URL
url = 'http://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios'.format(estado)
url

In [ ]:
df = pd.read_json(url)

In [ ]:
# Seleciona Colunas
df = df[['id', 'nome']]

# Renomeia Colunas
df = df.rename(columns=lambda x: x.replace('id', 'id_ibge'))
df['id_ibge'] = str(df['id_ibge'])
df.dtypes

In [ ]:
# Geojson file
json_file = os.path.join('data', 'divadmin.json')

# Geodataframe
gdf = gpd.read_file(json_file)
#gdf['codarea'] = pd.to_numeric(gdf['codarea'])
gdf

In [ ]:
#set(gdf['codarea'])
gdf.dtypes

In [ ]:
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=8,
)

In [ ]:
folium.Choropleth(
    geo_data=json_file,
    #geo_data=gdf,
    name='choropleth',    
    data=df,
    columns=['id_ibge', 'nome'],
    key_on='features.properties.codearea', # Tá aqui o erro
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate %',
).add_to(m)

m

In [ ]:
json.load(open(json_file))

In [ ]:
import requests
req = requests.get('https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json')
geofile = req.json()

In [ ]:
geofile